In [116]:
import pandas as pd
from datetime import datetime
import bcrypt

In [123]:
now = datetime.now()
dt_string = now.strftime("%Y-%m-%d %H:%M:%S")

In [141]:
df = pd.read_csv('csv/users.csv', encoding='utf-8')

In [142]:
image = pd.read_csv('out/image.csv', encoding='utf-8')

In [143]:
df = df.drop(['21.5.2022', 'Profile picture', 'Review1', 'Review2', 'Review3', 'Review4', 'Review5', 'Review6', 'Shutterstock URL', 'Photo ID'], axis = 1)

In [144]:
df.rename(columns = {'PUBLIC':'hasPublicProfile', 'Nationality' : 'nationality', \
                    'Age' : 'age', 'Sex' : 'sex', 'About Me' : 'bio', 'Name' : 'firstName', \
                    'Surname' : 'lastName', 'Username' : 'username'}, inplace = True)

In [145]:
columns = ['id', 'createdDate', 'updatedDate', 'isActive', 'email', 'hash', 'hasPaswordSaved', \
          'phone', 'statusId', 'defaultLocalityId', 'coverImageId', 'level', 'passwordResetHash', 'passwordResetValid', \
          'searchVector', 'searchConfig', 'defaultCurrencyId', 'imageId']

In [146]:
df = pd.concat([df, pd.DataFrame(columns = columns)])

In [147]:
df.head()

,hasPublicProfile,nationality,age,sex,bio,firstName,lastName,username,Photo name,new_name,...,statusId,defaultLocalityId,coverImageId,level,passwordResetHash,passwordResetValid,searchVector,searchConfig,defaultCurrencyId,imageId
0,ANO,United Kingdom,26.0,Male,"Hello there, this is Bruno, I'm a fullstack so...",Bruno,Turner,BrunoTurner123,1_773704330,4330BrunoTurner123,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ANO,United Kingdom,32.0,Male,"I'm Noel, a dedicaded sex and sexuality based ...",Noel,James,JamesN,2_1105379816,9816JamesN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ANO,United Kingdom,30.0,Male,"Hi, this is Abel, I'm an international investo...",Abel,Lee,Alee,3_576223936,3936Alee,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ANO,United Kingdom,24.0,Male,"In my childhood days, my love for observing th...",Seth,Scott,Scott007,4_326160794,0794Scott007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ANO,United Kingdom,25.0,Male,My collegues call me Chief Chef Aaron although...,Aaron,Hunter,LittleHunter,5_1127040110,0110LittleHunter,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [148]:
def set_public_profile(value):
    if value == 'ANO':
        return True
    return False

In [128]:
def create_pass(value):
    temp = 'Trynego+' + str(value).lower()
    bytePwd = temp.encode('utf-8')
    mySalt = bcrypt.gensalt()
    hash = bcrypt.hashpw(bytePwd, mySalt)
    return hash

In [149]:
df['id'] = df.index + 1
df['order'] = ''
df['createdDate'] = dt_string
df['updatedDate'] = dt_string
df['hasPublicProfile'] = df['hasPublicProfile'].apply(set_public_profile)
df['age'] = df.age.astype(int)
df['sex'] = df.sex.str.lower()
df['statusId'] = 4
df['level'] = 1

In [130]:
df['hash'] = df.username.apply(create_pass)

In [152]:
df['isActive'] = True
df['email'] = df.firstName.str.lower() + '.' + df.lastName.str.lower() + '@trynego.com'

In [153]:
def set_image_id(value):
    try:
        return image[:][image['name'] == str(value)]['id'].values[0]
    except:
        return ''

In [154]:
df['imageId'] = df['Photo name'].apply(set_image_id)

In [155]:
df = df.drop(['Photo name'], axis = 1)

In [156]:
df.head()

,hasPublicProfile,nationality,age,sex,bio,firstName,lastName,username,new_name,id,...,defaultLocalityId,coverImageId,level,passwordResetHash,passwordResetValid,searchVector,searchConfig,defaultCurrencyId,imageId,order
0,True,United Kingdom,26,male,"Hello there, this is Bruno, I'm a fullstack so...",Bruno,Turner,BrunoTurner123,4330BrunoTurner123,1,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,,
1,True,United Kingdom,32,male,"I'm Noel, a dedicaded sex and sexuality based ...",Noel,James,JamesN,9816JamesN,2,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,,
2,True,United Kingdom,30,male,"Hi, this is Abel, I'm an international investo...",Abel,Lee,Alee,3936Alee,3,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,634,
3,True,United Kingdom,24,male,"In my childhood days, my love for observing th...",Seth,Scott,Scott007,0794Scott007,4,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,,
4,True,United Kingdom,25,male,My collegues call me Chief Chef Aaron although...,Aaron,Hunter,LittleHunter,0110LittleHunter,5,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,,


In [157]:
df.to_csv('out/users.csv', index=False, encoding='utf-8')